In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from catboost import CatBoostRegressor

from training_CBR import predictions

In [7]:
# Load Training Data
train_df = pd.read_csv("./inputs/train.csv").set_index("id")
target = train_df["Listening_Time_minutes"]
train_df.drop("Listening_Time_minutes", axis=1, inplace=True)
train_df

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
id,,,,,,,,,,
0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive
1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative
2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative
3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive
4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral
...,...,...,...,...,...,...,...,...,...,...
749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative
749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral
749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative


In [8]:
# Load Testing Data
test_df = pd.read_csv("./inputs/test.csv").set_index("id")
test_df

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
id,,,,,,,,,,
750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral
...,...,...,...,...,...,...,...,...,...,...
999995,Mind & Body,Episode 100,21.05,Health,65.77,Saturday,Evening,96.40,3.0,Negative
999996,Joke Junction,Episode 85,85.50,Comedy,41.47,Saturday,Night,30.52,2.0,Negative
999997,Joke Junction,Episode 63,12.11,Comedy,25.92,Thursday,Evening,73.69,1.0,Neutral


In [9]:
# Encode the training categorical columns
%store -r categories
display(categories)
encoder = LabelEncoder()
for column in categories:
    train_df[column] = encoder.fit_transform(train_df[column])

train_df[categories]

['Podcast_Name',
 'Episode_Title',
 'Genre',
 'Publication_Day',
 'Publication_Time',
 'Episode_Sentiment']

,Podcast_Name,Episode_Title,Genre,Publication_Day,Publication_Time,Episode_Sentiment
id,,,,,,
0,34,98,9,4,3,2
1,24,19,1,2,0,0
2,40,8,2,5,1,0
3,10,40,8,1,2,2
4,31,85,3,1,0,1
...,...,...,...,...,...,...
749995,26,18,2,2,2,0
749996,2,14,0,2,3,1
749997,28,47,4,4,2,0


In [12]:
# Encode the testing categorical columns
%store -r categories
display(categories)
encoder = LabelEncoder()
for column in categories:
    test_df[column] = encoder.fit_transform(test_df[column])

test_df[categories]

['Podcast_Name',
 'Episode_Title',
 'Genre',
 'Publication_Day',
 'Publication_Time',
 'Episode_Sentiment']

,Podcast_Name,Episode_Title,Genre,Publication_Day,Publication_Time,Episode_Sentiment
id,,,,,,
750000,11,71,2,2,1,1
750001,36,16,5,3,2,1
750002,24,3,1,0,1,2
750003,4,71,1,3,2,2
750004,27,46,4,6,2,1
...,...,...,...,...,...,...
999995,31,2,3,2,1,0
999996,24,84,1,2,3,0
999997,24,60,1,4,1,1


In [15]:
# Scale the bot training and testing data
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_df)
scaled_test = scaler.transform(test_df)

In [16]:
X_train = scaled_train
X_test = scaled_test
y_train = target

In [19]:
# Using CatBoost Regressor
model = CatBoostRegressor()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
predictions

Learning rate set to 0.11653
0:	learn: 24.8877296	total: 55.3ms	remaining: 55.3s
1:	learn: 22.9678128	total: 113ms	remaining: 56.3s
2:	learn: 21.3199561	total: 173ms	remaining: 57.4s
3:	learn: 19.9166178	total: 232ms	remaining: 57.8s
4:	learn: 18.7565765	total: 292ms	remaining: 58.1s
5:	learn: 17.7726218	total: 350ms	remaining: 58s
6:	learn: 16.9491783	total: 419ms	remaining: 59.4s
7:	learn: 16.2725466	total: 471ms	remaining: 58.5s
8:	learn: 15.7055039	total: 523ms	remaining: 57.6s
9:	learn: 15.2375698	total: 577ms	remaining: 57.1s
10:	learn: 14.8611519	total: 646ms	remaining: 58.1s
11:	learn: 14.5521660	total: 723ms	remaining: 59.5s
12:	learn: 14.3006506	total: 785ms	remaining: 59.6s
13:	learn: 14.1001233	total: 847ms	remaining: 59.7s
14:	learn: 13.9368600	total: 910ms	remaining: 59.8s
15:	learn: 13.8028248	total: 962ms	remaining: 59.2s
16:	learn: 13.6951784	total: 1.01s	remaining: 58.5s
17:	learn: 13.6074851	total: 1.07s	remaining: 58.3s
18:	learn: 13.5382070	total: 1.14s	remaining: 

array([55.92900917, 18.11109784, 49.97091354, ...,  5.42481774,
       72.22579444, 57.80660972])

In [18]:
submission_test = pd.read_csv("./inputs/test.csv")
output = pd.DataFrame({'id': submission_test.id, 'Listening_Time_minutes': predictions})
output.to_csv('./outputs/submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [20]:
# Using Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
predictions

array([57.8317239, 23.4721651, 50.4865636, ...,  8.4007559, 72.4988788,
       57.9065444])

In [21]:
submission_test = pd.read_csv("./inputs/test.csv")
output = pd.DataFrame({'id': submission_test.id, 'Listening_Time_minutes': predictions})
output.to_csv('./outputs/submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
